# Essa é base que usaremos (import e pip)

In [ ]:
%pip install -q torch torchvision

In [ ]:
import torch                                # Importa a biblioteca base do PyTorch para criar e manipular tensores
import torch.nn as nn                       # Traz os "blocos de construção" de redes neurais
import torch.optim as optim                 # Importa os algoritmos de otimização que atualizam os pesos da rede.
import torchvision                          # Biblioteca auxiliar focada em Visão Computacional
import torchvision.transforms as transforms # Ferramentas para processar, redimensionar e transformar imagens antes de entrarem na rede
from torch.utils.data import DataLoader     # Utilitário que gerencia o carregamento dos dados em pequenos batches de forma eficiente.
import matplotlib.pyplot as plt             # Auxiliar na visualização
import numpy as np                          # Base de cálculos para a visualização


# Arquitetura da Rede Neural

In [ ]:
# Criar uma classe que herda o nn.module que é a base dos modelos em pytorch

class AlexNetSimplificada(nn.Module):

    # Construtor que vai ser rodado quando a class for instanciada

    def __init__(self, num_classes=10):

        # Chama o construtor da classe pai

        super(AlexNetSimplificada, self).__init__()

        """
        in_channels : quantidade de canais de entrada.
        out_channels: quantidade de canais de saída
        kernel_size: Tamanho de uma matriz x por x
        stride: anda x pixels por vez
        padding: adiciona x pixels de borda (zeros) em volta da imagem para controlar o tamanho da saída.
        ReLU (Rectified Linear Unit): zera valores negativos e mantém os positivos.
        """
        
        self.features = nn.Sequential(

            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(256 * 6 * 6, 4096), 
            nn.ReLU(inplace=True),
            
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            
            nn.Linear(4096, num_classes),
        )

# Define como os dados passam pela rede

    def forward(self, x):
  
        x = self.features(x)
        
        x = torch.flatten(x, 1)
        
        x = self.classifier(x)
        return x

In [17]:
# Exemplo

def inspecionar_dimensoes():
    print("--- INICIANDO A PROVA DE CONCEITO (RAIO-X) ---")
    
    modelo = AlexNetSimplificada()
    
    x = torch.randn(1, 3, 224, 224)
    print(f"0. Imagem Original: \t\t{x.shape} (Muito espaço, pouca profundidade)")
 
    x = modelo.features[0](x)
    print(f"1. Após 1ª Convolução: \t{x.shape} (Note: caiu p/ 55x55, mas canais foram p/ 64)")
    
    x = modelo.features[1](x)
    x = modelo.features[2](x) 
    print(f"2. Após 1º Pooling: \t\t{x.shape} (Caiu pela metade espacialmente)")

    x = modelo.features[3](x) 
    print(f"3. Após 2ª Convolução: \t{x.shape} (Canais aumentaram para 192)")
    
    x = modelo.features[4](x) 
    x = modelo.features[5](x) 
    print(f"4. Após 2º Pooling: \t\t{x.shape} (Espaço reduzido de novo)")

    for i in range(6, 11): 
        x = modelo.features[i](x)
    print(f"5. Após Convoluções finais: \t{x.shape} (Profundidade máxima, espaço pequeno)")
    
    x = modelo.features[11](x) 
    print(f"6. Saída do Extrator (Features): {x.shape} -> ESTE É O CUBO FINAL 6x6")

    x = torch.flatten(x, 1)
    print(f"7. Após Flatten (Achatamento): \t{x.shape} -> (256 * 6 * 6 = 9216 números em linha)")

inspecionar_dimensoes()


--- INICIANDO A PROVA DE CONCEITO (RAIO-X) ---
0. Imagem Original: 		torch.Size([1, 3, 224, 224]) (Muito espaço, pouca profundidade)
1. Após 1ª Convolução: 	torch.Size([1, 64, 55, 55]) (Note: caiu p/ 55x55, mas canais foram p/ 64)
2. Após 1º Pooling: 		torch.Size([1, 64, 27, 27]) (Caiu pela metade espacialmente)
3. Após 2ª Convolução: 	torch.Size([1, 192, 27, 27]) (Canais aumentaram para 192)
4. Após 2º Pooling: 		torch.Size([1, 192, 13, 13]) (Espaço reduzido de novo)
5. Após Convoluções finais: 	torch.Size([1, 256, 13, 13]) (Profundidade máxima, espaço pequeno)
6. Saída do Extrator (Features): torch.Size([1, 256, 13, 13]) -> ESTE É O CUBO FINAL 6x6
7. Após Flatten (Achatamento): 	torch.Size([1, 43264]) -> (256 * 6 * 6 = 9216 números em linha)


In [ ]:
BATCH_SIZE = 64        
LEARNING_RATE = 0.001  
EPOCHS = 5             
NUM_CLASSES = 10       

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),              
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) 
])

print("Baixando e preparando dados...")

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                           download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

model = AlexNetSimplificada(num_classes=NUM_CLASSES) 
model = model.to(device) 


criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("Iniciando treinamento... isso pode demorar dependendo do seu PC.")

for epoch in range(EPOCHS):
    running_loss = 0.0
    
    for i, (images, labels) in enumerate(train_loader):

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)

        loss = criterion(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if (i+1) % 100 == 0: 
            print(f"Época [{epoch+1}/{EPOCHS}], Passo [{i+1}/{len(train_loader)}], Perda: {running_loss/100:.4f}")
            running_loss = 0.0

print("Treinamento concluído!")

In [ ]:
def calcular_acuracia():
    print("Calculando acurácia no conjunto de TESTE (imagens que a rede nunca viu)...")
    
    # Prepara o dataset de teste (igual ao de treino, mas train=False)
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                              download=True, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

    correct = 0
    total = 0
    
    # Coloca em modo de avaliação (desliga Dropout, etc)
    model.eval() 
    
    with torch.no_grad(): # Não precisamos calcular gradientes agora
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            
            # Pega a classe com maior probabilidade
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f'Acurácia da rede em 10.000 imagens de teste: {acc:.2f}%')
    
    # Volta para modo de treino caso queira treinar mais
    model.train()

calcular_acuracia()

In [ ]:


def mostrar_visualmente():
    # Classes do CIFAR-10
    classes = ('avião', 'carro', 'pássaro', 'gato', 'cervo', 
               'cachorro', 'sapo', 'cavalo', 'navio', 'caminhão')

    # Pega um batch de imagens de teste
    test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                              download=True, transform=transform)
    loader = DataLoader(test_dataset, batch_size=5, shuffle=True)
    images, labels = next(iter(loader))
    
    # Move para GPU para fazer a previsão
    images_device = images.to(device)
    model.eval()
    outputs = model(images_device)
    _, predicted = torch.max(outputs, 1)

    # Configuração do plot
    fig = plt.figure(figsize=(15, 5))
    
    for i in range(5):
        ax = fig.add_subplot(1, 5, i+1)
        
        # Desnormalizar a imagem para mostrar (senão fica escura/estranha)
        img = images[i] / 2 + 0.5     # desfaz a normalização
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0))) # Converte de (C,H,W) para (H,W,C)
        
        # Cor do título: Verde se acertou, Vermelho se errou
        cor = 'green' if predicted[i] == labels[i] else 'red'
        
        titulo = f"Real: {classes[labels[i]]}\nPred: {classes[predicted[i]]}"
        ax.set_title(titulo, color=cor)
        ax.axis('off')
    
    plt.show()

mostrar_visualmente()

In [ ]:
# Salva apenas os pesos (state_dict) - é o jeito recomendado
caminho_arquivo = 'alexnet_cifar10_custom.pth'
torch.save(model.state_dict(), caminho_arquivo)
print(f"Modelo salvo com sucesso em: {caminho_arquivo}")

# Dica: Se estiver no Google Colab, faça o download para o seu PC
try:
    from google.colab import files
    files.download(caminho_arquivo)
except:
    print("Arquivo salvo localmente. Busque na pasta de arquivos.")